# **Trajectory**

**Detection**

From the Detection phase I want to have:
|What I want|Where it should be|
|-----------|------------------|
|The video with the detection|*.../data/recording...*|
|Data about the posizion of the ball for each frame|*.../data/auxiliary_data/Circle_posizions_clean.csv*|
|(optional) The video only of the ball detected|*.../data/recording...*|

If not present the file */data/auxiliary_data/Circle_positions.csv* run the Detection code at */notebook/ball_detection/Ball_detection_analysis.ipynb* in order to produce it

**Trajectory computation**

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import csv

Loading files paths

In [ ]:
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_2" / "Recording_2_normal_speed.mp4")
CSV_FILE_PATH = str(PROJECT_ROOT / "data" / "auxiliary_data" / "Circle_positions_4_clean.csv")
OUTPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_4" / "Tracked_output_4.mp4")

Creating video *Tracked_output.mp4*

In [30]:
# Load video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
    
# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

In [31]:
# Read circle positions from CSV
positions = {}
with open(CSV_FILE_PATH, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        frame_num, x, y = row
        positions[int(frame_num)] = (int(x), int(y)) if x and y else None

In [32]:
frame_count = 0
trajectory = []  # Store trajectory points
    
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Draw tracking path
    if frame_count in positions and positions[frame_count] is not None:
        x, y = positions[frame_count]
        trajectory.append((x, y))
    
    # Draw trajectory
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i - 1], trajectory[i], (0, 255, 255), 2)
    
    # Draw current ball position
    if frame_count in positions and positions[frame_count] is not None:
        x, y = positions[frame_count]
        cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)  # Red dot for current position
    
    out.write(frame)
    frame_count += 1
    
cap.release()
out.release()
print(f"Tracking video saved to {OUTPUT_VIDEO_PATH}")

Tracking video saved to /home/davic/projects/IACV_project/bowling-analysis/data/recording_4/Tracked_output_4.mp4


Having the reconstruction of the plane I can trasform the coordinates in the other system.